# Transfer Learning on DenseNet121

In [1]:
import tensorflow as tf

file_dir = tf.keras.utils.get_file(fname='flower_photos',
                                   origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                   untar=True)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [2]:
import os

train_set, val_set = tf.keras.utils.image_dataset_from_directory(os.path.join(file_dir, 'flower_photos'),
                                                                 validation_split=0.2,
                                                                 subset='both',
                                                                 seed=420)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Using 734 files for validation.


In [3]:
def data_augment(img, label):
  img = tf.image.random_brightness(img, 0.1)
  img = tf.image.random_contrast(img, 0.9, 1.1)
  img = tf.image.random_flip_left_right(img)
  img = tf.image.random_flip_up_down(img)

  return img, label

In [4]:
def data_prep(img, label):
  img = tf.keras.applications.densenet.preprocess_input(img)

  return img, label

In [5]:
train_set = train_set.map(data_prep)
train_set = train_set.map(data_augment)
train_set = train_set.prefetch(tf.data.AUTOTUNE)

val_set = val_set.map(data_prep)
val_set = val_set.cache().prefetch(tf.data.AUTOTUNE)

In [6]:
base_model =tf.keras.applications.DenseNet121(include_top=False,
                                              weights="imagenet",
                                              pooling='avg')
base_model.trainable = False

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

model = Sequential([
    base_model,
    Flatten(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ (None, 1024)           │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,042,629 (26.87 MB)

 Trainable params: 5,125 (20.02 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_set, batch_size=32, epochs=40, validation_data=val_set)

Epoch 1/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 109s 852ms/step - accuracy: 0.2509 - loss: 2.0653 - val_accuracy: 0.4114 - val_loss: 1.4694
Epoch 2/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 68s 223ms/step - accuracy: 0.3307 - loss: 1.7482 - val_accuracy: 0.5286 - val_loss: 1.2628
Epoch 3/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 224ms/step - accuracy: 0.4053 - loss: 1.5335 - val_accuracy: 0.6008 - val_loss: 1.1047
Epoch 4/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 220ms/step - accuracy: 0.4461 - loss: 1.4098 - val_accuracy: 0.6676 - val_loss: 0.9763
Epoch 5/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 229ms/step - accuracy: 0.5225 - loss: 1.2157 - val_accuracy: 0.7071 - val_loss: 0.8773
Epoch 6/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 215ms/step - accuracy: 0.5778 - loss: 1.1214 - val_accuracy: 0.7425 - val_loss: 0.7998
Epoch 7/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 224ms/step - accuracy: 0.5946 - loss: 1.0387 - val_accuracy: 0.7779 - val_loss: 0.7359
Epoch 8/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 213ms/step - accuracy: 0.6389 - loss: 0.9905 - val_acc

In [11]:
model.evaluate(train_set)

92/92 ━━━━━━━━━━━━━━━━━━━━ 25s 267ms/step - accuracy: 0.9107 - loss: 0.3065


[0.3126412034034729, 0.9100817441940308]

# Fine Tuning:

In [12]:
base_model.summary()

Model: "densenet121"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,408 │ zero_padding2d[0… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, None,      │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, None,      │          0 │ conv2_block1_0_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, None,      │          0 │ pool1[0][0],      │
│ (Concatenate)       │ None, 96)         │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, None,      │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, None,      │          0 │ conv2_block2_0_b… │
│ (Activation)        │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, None,      │     12,288 │ conv2_block2_0_r

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,037,504 (26.85 MB)

Unfreezing later layers.

In [13]:
is_seen = False
for l in base_model.layers:
  if l.name == 'conv5_block14_0_bn':
    is_seen = True
  if is_seen == True:
    l.trainable = True
    print(l.name)

conv5_block14_0_bn
conv5_block14_0_relu
conv5_block14_1_conv
conv5_block14_1_bn
conv5_block14_1_relu
conv5_block14_2_conv
conv5_block14_concat
conv5_block15_0_bn
conv5_block15_0_relu
conv5_block15_1_conv
conv5_block15_1_bn
conv5_block15_1_relu
conv5_block15_2_conv
conv5_block15_concat
conv5_block16_0_bn
conv5_block16_0_relu
conv5_block16_1_conv
conv5_block16_1_bn
conv5_block16_1_relu
conv5_block16_2_conv
conv5_block16_concat
bn
relu
avg_pool


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ (None, 1024)           │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,052,881 (26.90 MB)

 Trainable params: 492,933 (1.88 MB)

 Non-trainable params: 6,549,696 (24.99 MB)

 Optimizer params: 10,252 (40.05 KB)

In [15]:
model.fit(train_set, batch_size=32, epochs=60, validation_data=val_set, initial_epoch=40)

Epoch 41/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 210ms/step - accuracy: 0.8475 - loss: 0.4508 - val_accuracy: 0.8924 - val_loss: 0.3279
Epoch 42/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 216ms/step - accuracy: 0.8357 - loss: 0.4636 - val_accuracy: 0.8910 - val_loss: 0.3245
Epoch 43/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 210ms/step - accuracy: 0.8415 - loss: 0.4394 - val_accuracy: 0.8937 - val_loss: 0.3228
Epoch 44/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 220ms/step - accuracy: 0.8423 - loss: 0.4342 - val_accuracy: 0.8924 - val_loss: 0.3238
Epoch 45/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 212ms/step - accuracy: 0.8538 - loss: 0.4301 - val_accuracy: 0.8937 - val_loss: 0.3212
Epoch 46/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 211ms/step - accuracy: 0.8424 - loss: 0.4553 - val_accuracy: 0.8951 - val_loss: 0.3175
Epoch 47/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 204ms/step - accuracy: 0.8531 - loss: 0.4322 - val_accuracy: 0.8937 - val_loss: 0.3143
Epoch 48/60
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 217ms/step - accuracy: 0.8377 - loss: 0.4710 - 

Reducing learning rate.

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, batch_size=32, epochs=70, validation_data=val_set, initial_epoch=60)

Epoch 61/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 87s 569ms/step - accuracy: 0.8172 - loss: 0.5326 - val_accuracy: 0.8978 - val_loss: 0.2869
Epoch 62/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 220ms/step - accuracy: 0.8412 - loss: 0.4319 - val_accuracy: 0.9033 - val_loss: 0.2756
Epoch 63/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 217ms/step - accuracy: 0.8534 - loss: 0.4272 - val_accuracy: 0.9019 - val_loss: 0.2675
Epoch 64/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 234ms/step - accuracy: 0.8543 - loss: 0.4087 - val_accuracy: 0.9019 - val_loss: 0.2612
Epoch 65/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 222ms/step - accuracy: 0.8754 - loss: 0.3644 - val_accuracy: 0.9046 - val_loss: 0.2555
Epoch 66/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 230ms/step - accuracy: 0.8596 - loss: 0.3883 - val_accuracy: 0.9142 - val_loss: 0.2482
Epoch 67/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 229ms/step - accuracy: 0.8705 - loss: 0.3675 - val_accuracy: 0.9142 - val_loss: 0.2427
Epoch 68/70
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 225ms/step - accuracy: 0.8670 - loss: 0.3872 - 

Unfreezing more layers.

In [17]:
is_seen = False
for l in base_model.layers:
  if l.name == 'conv5_block12_0_bn':
    is_seen = True
  if is_seen == True:
    l.trainable = True
    print(l.name)

conv5_block12_0_bn
conv5_block12_0_relu
conv5_block12_1_conv
conv5_block12_1_bn
conv5_block12_1_relu
conv5_block12_2_conv
conv5_block12_concat
conv5_block13_0_bn
conv5_block13_0_relu
conv5_block13_1_conv
conv5_block13_1_bn
conv5_block13_1_relu
conv5_block13_2_conv
conv5_block13_concat
conv5_block14_0_bn
conv5_block14_0_relu
conv5_block14_1_conv
conv5_block14_1_bn
conv5_block14_1_relu
conv5_block14_2_conv
conv5_block14_concat
conv5_block15_0_bn
conv5_block15_0_relu
conv5_block15_1_conv
conv5_block15_1_bn
conv5_block15_1_relu
conv5_block15_2_conv
conv5_block15_concat
conv5_block16_0_bn
conv5_block16_0_relu
conv5_block16_1_conv
conv5_block16_1_bn
conv5_block16_1_relu
conv5_block16_2_conv
conv5_block16_concat
bn
relu
avg_pool


In [18]:
model.fit(train_set, batch_size=32, epochs=80, validation_data=val_set, initial_epoch=70)

Epoch 71/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 226ms/step - accuracy: 0.8868 - loss: 0.3203 - val_accuracy: 0.9169 - val_loss: 0.2277
Epoch 72/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 224ms/step - accuracy: 0.8811 - loss: 0.3345 - val_accuracy: 0.9114 - val_loss: 0.2242
Epoch 73/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 205ms/step - accuracy: 0.8991 - loss: 0.2884 - val_accuracy: 0.9169 - val_loss: 0.2197
Epoch 74/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 223ms/step - accuracy: 0.8928 - loss: 0.2824 - val_accuracy: 0.9169 - val_loss: 0.2161
Epoch 75/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 219ms/step - accuracy: 0.8968 - loss: 0.2775 - val_accuracy: 0.9196 - val_loss: 0.2135
Epoch 76/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 213ms/step - accuracy: 0.8994 - loss: 0.2701 - val_accuracy: 0.9223 - val_loss: 0.2104
Epoch 77/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 225ms/step - accuracy: 0.8897 - loss: 0.2888 - val_accuracy: 0.9223 - val_loss: 0.2073
Epoch 78/80
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 209ms/step - accuracy: 0.8937 - loss: 0.2690 - 

Reducing learning rate.

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, batch_size=32, epochs=90, validation_data=val_set, initial_epoch=80)

Epoch 81/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 98s 652ms/step - accuracy: 0.9198 - loss: 0.2389 - val_accuracy: 0.9210 - val_loss: 0.2027
Epoch 82/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 218ms/step - accuracy: 0.9137 - loss: 0.2363 - val_accuracy: 0.9210 - val_loss: 0.2026
Epoch 83/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 221ms/step - accuracy: 0.9123 - loss: 0.2520 - val_accuracy: 0.9210 - val_loss: 0.2010
Epoch 84/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 219ms/step - accuracy: 0.9127 - loss: 0.2389 - val_accuracy: 0.9210 - val_loss: 0.1994
Epoch 85/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 43s 239ms/step - accuracy: 0.9085 - loss: 0.2509 - val_accuracy: 0.9210 - val_loss: 0.1981
Epoch 86/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 39s 219ms/step - accuracy: 0.9087 - loss: 0.2307 - val_accuracy: 0.9196 - val_loss: 0.1973
Epoch 87/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 226ms/step - accuracy: 0.9095 - loss: 0.2470 - val_accuracy: 0.9210 - val_loss: 0.1975
Epoch 88/90
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 230ms/step - accuracy: 0.9188 - loss: 0.2106 - 

Unfreezing more layers.

In [20]:
is_seen = False
for l in base_model.layers:
  if l.name == 'conv5_block12_0_bn':
    is_seen = True
  if is_seen == True:
    l.trainable = True
    print(l.name)

conv5_block12_0_bn
conv5_block12_0_relu
conv5_block12_1_conv
conv5_block12_1_bn
conv5_block12_1_relu
conv5_block12_2_conv
conv5_block12_concat
conv5_block13_0_bn
conv5_block13_0_relu
conv5_block13_1_conv
conv5_block13_1_bn
conv5_block13_1_relu
conv5_block13_2_conv
conv5_block13_concat
conv5_block14_0_bn
conv5_block14_0_relu
conv5_block14_1_conv
conv5_block14_1_bn
conv5_block14_1_relu
conv5_block14_2_conv
conv5_block14_concat
conv5_block15_0_bn
conv5_block15_0_relu
conv5_block15_1_conv
conv5_block15_1_bn
conv5_block15_1_relu
conv5_block15_2_conv
conv5_block15_concat
conv5_block16_0_bn
conv5_block16_0_relu
conv5_block16_1_conv
conv5_block16_1_bn
conv5_block16_1_relu
conv5_block16_2_conv
conv5_block16_concat
bn
relu
avg_pool


In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, batch_size=32, epochs=100, validation_data=val_set, initial_epoch=90)

Epoch 91/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 88s 563ms/step - accuracy: 0.9336 - loss: 0.2045 - val_accuracy: 0.9223 - val_loss: 0.1934
Epoch 92/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 234ms/step - accuracy: 0.9056 - loss: 0.2337 - val_accuracy: 0.9223 - val_loss: 0.1927
Epoch 93/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 229ms/step - accuracy: 0.9294 - loss: 0.1914 - val_accuracy: 0.9223 - val_loss: 0.1917
Epoch 94/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 225ms/step - accuracy: 0.9398 - loss: 0.1976 - val_accuracy: 0.9223 - val_loss: 0.1916
Epoch 95/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 225ms/step - accuracy: 0.9322 - loss: 0.2082 - val_accuracy: 0.9251 - val_loss: 0.1902
Epoch 96/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 208ms/step - accuracy: 0.9267 - loss: 0.1983 - val_accuracy: 0.9251 - val_loss: 0.1896
Epoch 97/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 227ms/step - accuracy: 0.9265 - loss: 0.2081 - val_accuracy: 0.9251 - val_loss: 0.1884
Epoch 98/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 214ms/step - accuracy: 0.9411 - loss: 0

Increasing learning rate.

In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, batch_size=32, epochs=105, validation_data=val_set, initial_epoch=100)

Epoch 101/105
92/92 ━━━━━━━━━━━━━━━━━━━━ 88s 576ms/step - accuracy: 0.9292 - loss: 0.1877 - val_accuracy: 0.9292 - val_loss: 0.1853
Epoch 102/105
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 224ms/step - accuracy: 0.9304 - loss: 0.1849 - val_accuracy: 0.9319 - val_loss: 0.1812
Epoch 103/105
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 234ms/step - accuracy: 0.9453 - loss: 0.1600 - val_accuracy: 0.9387 - val_loss: 0.1796
Epoch 104/105
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 224ms/step - accuracy: 0.9596 - loss: 0.1159 - val_accuracy: 0.9360 - val_loss: 0.1729
Epoch 105/105
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 224ms/step - accuracy: 0.9617 - loss: 0.1162 - val_accuracy: 0.9319 - val_loss: 0.1756


Decreasing learning rate.

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, batch_size=32, epochs=110, validation_data=val_set, initial_epoch=105)

Epoch 106/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 88s 577ms/step - accuracy: 0.9706 - loss: 0.0919 - val_accuracy: 0.9332 - val_loss: 0.1720
Epoch 107/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 226ms/step - accuracy: 0.9630 - loss: 0.1235 - val_accuracy: 0.9346 - val_loss: 0.1715
Epoch 108/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 231ms/step - accuracy: 0.9598 - loss: 0.1106 - val_accuracy: 0.9346 - val_loss: 0.1721
Epoch 109/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 39s 214ms/step - accuracy: 0.9729 - loss: 0.0855 - val_accuracy: 0.9360 - val_loss: 0.1700
Epoch 110/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 232ms/step - accuracy: 0.9673 - loss: 0.0874 - val_accuracy: 0.9387 - val_loss: 0.1689


In [24]:
model.fit(train_set, batch_size=32, epochs=115, validation_data=val_set, initial_epoch=110)

Epoch 111/115
92/92 ━━━━━━━━━━━━━━━━━━━━ 24s 251ms/step - accuracy: 0.9679 - loss: 0.0902 - val_accuracy: 0.9360 - val_loss: 0.1660
Epoch 112/115
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 232ms/step - accuracy: 0.9646 - loss: 0.0911 - val_accuracy: 0.9360 - val_loss: 0.1679
Epoch 113/115
92/92 ━━━━━━━━━━━━━━━━━━━━ 23s 242ms/step - accuracy: 0.9779 - loss: 0.0763 - val_accuracy: 0.9387 - val_loss: 0.1708
Epoch 114/115
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 240ms/step - accuracy: 0.9714 - loss: 0.0890 - val_accuracy: 0.9387 - val_loss: 0.1715
Epoch 115/115
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 228ms/step - accuracy: 0.9711 - loss: 0.0821 - val_accuracy: 0.9360 - val_loss: 0.1704


In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, batch_size=32, epochs=110, validation_data=val_set, initial_epoch=105)

Epoch 106/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 92s 607ms/step - accuracy: 0.9781 - loss: 0.0662 - val_accuracy: 0.9373 - val_loss: 0.1694
Epoch 107/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 32s 231ms/step - accuracy: 0.9827 - loss: 0.0670 - val_accuracy: 0.9373 - val_loss: 0.1690
Epoch 108/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 231ms/step - accuracy: 0.9787 - loss: 0.0639 - val_accuracy: 0.9373 - val_loss: 0.1688
Epoch 109/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 226ms/step - accuracy: 0.9763 - loss: 0.0711 - val_accuracy: 0.9387 - val_loss: 0.1677
Epoch 110/110
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 220ms/step - accuracy: 0.9815 - loss: 0.0646 - val_accuracy: 0.9387 - val_loss: 0.1678
